<a href="https://colab.research.google.com/github/108213043/stonks/blob/main/w11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

In [3]:
import pandas as pd

# 加載訓練數據
news_data_path = os.path.join('/content/drive/MyDrive/智慧醫療/stock-market-prediction-and-sentimental-analysis/Combined_News_DJIA(train).csv')
stock_data_path = os.path.join('/content/drive/MyDrive/智慧醫療/stock-market-prediction-and-sentimental-analysis/DJIA_table(train).csv')

news_data = pd.read_csv(news_data_path)
stock_data = pd.read_csv(stock_data_path)

# 查看數據的前幾行
news_data.head(), stock_data.head()

(         Date  Label                                               Top1  \
 0  2008-08-08      0  b"Georgia 'downs two Russian warplanes' as cou...   
 1  2008-08-11      1  b'Why wont America and Nato help us? If they w...   
 2  2008-08-12      0  b'Remember that adorable 9-year-old who sang a...   
 3  2008-08-13      0  b' U.S. refuses Israel weapons to attack Iran:...   
 4  2008-08-14      1  b'All the experts admit that we should legalis...   
 
                                                 Top2  \
 0            b'BREAKING: Musharraf to be impeached.'   
 1        b'Bush puts foot down on Georgian conflict'   
 2                 b"Russia 'ends Georgia operation'"   
 3  b"When the president ordered to attack Tskhinv...   
 4  b'War in South Osetia - 89 pictures made by a ...   
 
                                                 Top3  \
 0  b'Russia Today: Columns of troops roll into So...   
 1  b"Jewish Georgian minister: Thanks to Israeli ...   
 2  b'"If we had no sexual 

In [4]:
# 數據處理：合併新聞與股價數據
# 格式化日期
news_data['Date'] = pd.to_datetime(news_data['Date'])
stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='%d-%m-%Y')

# 合併數據（內部合併）
merged_data = pd.merge(news_data, stock_data, on='Date', how='inner')

# 整理列名稱
merged_data = merged_data.rename(columns={'Label': 'Stock_Label'})

# 檢視合併後的數據
merged_data.head()


,Date,Stock_Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.08984,11759.95996,11388.04004,11734.32031,212830000,11734.32031
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.66992,11867.11035,11675.53027,11782.34961,183190000,11782.34961
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.70020,11782.34961,11601.51953,11642.46973,173590000,11642.46973
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,11632.80957,11633.78027,11453.33984,11532.95996,182550000,11532.95996
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Taliban wages war on humanitarian aid workers',"b'Russia: World ""can forget about"" Georgia\'s...",b'Darfur rebels accuse Sudan of mounting major...,b'Philippines : Peace Advocate say Muslims nee...,11532.07031,11718.28027,11450.88965,11615.92969,159790000,11615.92969


In [7]:
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 合併 Top1 至 Top25 的新聞文本為單一文本特徵
# merged_data["Combined_Text"] = merged_data.loc[:, "Top1":"Top2"].apply(
#     lambda row: " ".join(row.values.astype(str)), axis=1
# )
merged_data["Combined_Text"] = merged_data["Top1"]

# 分割數據集
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# 文本處理：使用 BERT 模型的 tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def process_texts(texts, tokenizer, max_len=128):
    tokens = tokenizer.batch_encode_plus(
        texts, max_length=max_len, padding=True, truncation=True, return_tensors="pt"
    )
    return tokens["input_ids"], tokens["attention_mask"]

# 提取股價數據的特徵
def process_stock_features(data):
    return torch.tensor(data[["Open", "High", "Low", "Close", "Volume"]].values, dtype=torch.float32)

# 模型結構：多模態分類
class MultimodalClassifier(nn.Module):
    def __init__(self, bert_model, stock_feature_size):
        super(MultimodalClassifier, self).__init__()
        self.bert = bert_model
        self.bert_fc = nn.Linear(768, 128)
        self.stock_fc = nn.Linear(stock_feature_size, 128)
        self.classifier = nn.Linear(256, 2)

    def forward(self, text_ids, attention_mask, stock_features):
        # print(text_ids.shape)
        # print(stock_features.shape)
        bert_output = self.bert(text_ids, attention_mask=attention_mask).pooler_output
        bert_output = torch.relu(self.bert_fc(bert_output))
        stock_output = torch.relu(self.stock_fc(stock_features))
        combined = torch.cat((bert_output, stock_output), dim=1)
        output = self.classifier(combined)
        return output

# 加載 BERT 模型
bert_model = AutoModel.from_pretrained("bert-base-uncased")
model = MultimodalClassifier(bert_model, stock_feature_size=5).to(device)

# 損失函數與優化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# 准備訓練與測試數據 並設定 batch size 32
batch_size = 32
def prepare_dataset(data, tokenizer):
    text_ids, attention_mask = process_texts(data["Combined_Text"].tolist(), tokenizer)
    stock_features = process_stock_features(data)
    labels = torch.tensor(data["Stock_Label"].values, dtype=torch.long)
    return text_ids.to(device), attention_mask.to(device), stock_features.to(device), labels.to(device)

train_text_ids, train_attention_mask, train_stock_features, train_labels = prepare_dataset(train_data, tokenizer)
test_text_ids, test_attention_mask, test_stock_features, test_labels = prepare_dataset(test_data, tokenizer)

# 訓練模型
# def train_model(model, criterion, optimizer, epochs=5):
#     model.train()
#     # 轉換成 batch 32
#     for epoch in range(epochs):
        # optimizer.zero_grad()
        # print(train_text_ids.shape)
        # print(train_attention_mask.shape)
        # print(train_stock_features.shape)
        # outputs = model(train_text_ids, train_attention_mask, train_stock_features)
        # loss = criterion(outputs, train_labels)
        # loss.backward()
        # optimizer.step()
        # print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
        # ... (其他程式碼維持不變)

# 訓練模型
def train_model(model, criterion, optimizer, epochs=5, batch_size=32):
    model.train()
    for epoch in range(epochs):
        # 建立資料載入器
        dataset = torch.utils.data.TensorDataset(train_text_ids, train_attention_mask, train_stock_features, train_labels)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for batch_idx, (text_ids, attention_mask, stock_features, labels) in enumerate(dataloader):
            optimizer.zero_grad()
            outputs = model(text_ids, attention_mask, stock_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {loss.item()}")

train_model(model, criterion, optimizer)

# 評估模型
def evaluate_model(model):
    model.eval()
    with torch.no_grad():
        outputs = model(test_text_ids, test_attention_mask, test_stock_features)
        predictions = torch.argmax(outputs, dim=1)
        print(classification_report(test_labels.cpu(), predictions.cpu()))

evaluate_model(model)

torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 1, Loss: 2701038.0
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 2, Loss: 1782245.125
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 3, Loss: 1624912.5
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 4, Loss: 1388656.625
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 5, Loss: 1540123.25
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 6, Loss: 1113606.375
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 7, Loss: 1237798.625
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 8, Loss: 854542.3125
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 9, Loss: 430963.625
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 10, Loss: 20245.28125
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 11, Loss: 310073.28125
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 12, Loss: 381130.84375
torch.Size([32, 79])
torch.Size([32, 5])
Epoch 1, Batch 13, Loss: 464900.8437

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
